# Chapter 1: Introduction

## A Lil' History

Before building up the notation and delving into bandit algorithms, 
it's worth thinking about where they came from and the problems 
that were initially considered when formulating the notion of 
bandit algorithms.

- 1933, William R. Thompson (medical trials, blindly finishing them)
- 1950s, Frederick Mosteller and Robert Bush (animal learning)

![Animal decision making](figs/animal.png)

The two-armed bandit machine!

![Two-Armed Bandit](figs/twoarms.png)

## Why Bandits?

- Simplistic adaptive learning scnearios
- Adaptive algorithms for web interfaces, recommendation algorithms, dynamic pricing
- Lot's of fun math (convex analysis, optimization, complexity, probability theory, information theory) 
- Growing body of literature, growing research interests
    - 2001-2005: <1000 papers
    - 2006-2010: 2700 papers
    - 2011-2015: 7000 papers
    - 2016-: 5600+ papers

## The Classic Dilemma

Say you have a slot machine:
- Pulling the left arm, you've won \$10 2/5 times 
- Pulling the right arm, you've won $10 1/5 times

Which arm should you keep pulling if you had 10 more times to pull?

Exploration versus explotation!! 

## Formalizing Notation

We are concerned how a **learner** interfaces with an **environment**.

Specifically, we will consider a sequential game played for $n$ rounds. 
This $n$ is called the horizon. 
A $t \in [n]$ is a specific round of game.

For each $t \in [n]$, our learner will select an action $A_t$ (in a set of possible actions $\mathcal{A}$).
This could be L or R if our choices are to select a left or right arm on a slot machine.
- Side note: "multi-arm" or "k-armed" bandits refer to the # of actions. For our previous dilemma, $k=2$ and we have a 2-armed bandit problem

At the end of each round, the environment reveals a reward: $X_t \in \mathbb{R}$. In the slot example, this may be money, but could be anything (like a binary 0/1 signal to denote whether a link or ad was clicked).

At time $t$, a learner has access to its **history**: $H_{t-1} = (A_1,X_1,...,A_{t-1},X_{t-1})$.
Obviously, at most, a learner only has access to the past and can never see the future.

A **policy** ($\pi$) is a map from a history to actions. That is, a policy is the way our learner will consider the past to inform the decision in makes for the current timestep.

A common (but not the only) goal is to maximize cumulative reward: $\sum_{t=1}^{n} X_{t}$.

### Environment? Environment Class?

At its core, the goal of a bandit problem is to design a learning algorithm that can function "well" in an unknown environment. 
In the most general sense, the term "unknown" may not be particularly useful.
For this reason, bandit problems are typically associated with what's called an **environment class** ($\mathcal{E}$).
This is a way to set a boundary around what our algorithms are expected to be able to learn.

### Regret

We've already highlighted one way of looking at algorithmic performance (the cumulative reward), but one of the major ones that anyone working with bandit algorithms _needs_ to be aware of is **regret**.

Here, we consider regret to be be a quantity defined between our learner and another policy $\pi$. 
It is the difference between the reward we expect our learner to get over $n$ rounds and the rewared expected from following policy $\pi$.

In fact, we can extend this notion to compare our learner against a set of different policies $\pi \in \Pi$.
Here, regret is defined as the _max_ regret attained by not following the best algorithm in the whole set. 
Another name for this set of policies is the **competitor class**.

Much like the name might suggest, this is how regretful we are that we did not use another policy when playing our game.

### Example 1.2, made more concrete

Example 1.2 in the text introduces a stochastic Bernoulli bandit problem to illustrate what regret might look like in practice.

Here, we have a bandit problem with $k$ arms where each arm has an associated [Bernoulli Distribution](https://en.wikipedia.org/wiki/Bernoulli_distribution),
each parameterized with 1 parameter $\mu_i, i \in \{1,...,k\}$ that defines the probability of generating a reward of 1.

The competitor class we will consider is the class of constant policies--AKA, the policies of only selecting one of the $k$ arms every time. 

This makes sense; if we knew the $mu_i$ a priori, we could simply select the arm with the highest average value and thus gain 0 regret.

Unfortunately, it is not typical for us to have that oracle knowledge. 

Out of curiosity, let's set $k=3$ and see how these constant algorithms would "fair" against one another.

In [19]:
import random

class BernoulliArm:
    
    '''
    Very basic Bernoulli Arm setup
    '''
    
    def __init__(self, mu):
        self._mu = mu
        
    def __call__(self):
        return 1 if random.random() <= self._mu else 0

In [30]:
# try out different probabilities!
mus = {
    0: 0.3,
    1: 0.35,
    2: 0.25,
}

bandit = {k: BernoulliArm(v) for k, v in mus.items()}

In [31]:
n = 2000 # set a time horizon

In [32]:
scores = {0: 0, 1: 0, 2: 0}
for t in range(n):
    for k, arm in bandit.items():
        scores[k] += arm()

# according to our expectation, arm 1 should have the highest reward!
scores

{0: 550, 1: 731, 2: 513}

In [34]:
# difference from "expected" probs. 
{k: n*mu for k, mu in mus.items()}

{0: 600.0, 1: 700.0, 2: 500.0}

In [35]:
regrets = {}
for k1, s1 in scores.items():
    max_regret = 0
    for k2, s2 in scores.items():
        d = s2 - s1
        if d > max_regret:
            max_regret = d
    regrets[k1] = max_regret

regrets

{0: 181, 1: 0, 2: 218}

## A Couple More Remarks on Problem Formulations 

- Worst-case regret, always
    - A "good" learner will achieve sub-linear (linear is never getting a positive reward) 
    - When do we achieve log(n)? sqrt(n)? 
    - How "good" is "good"?
- The more general the environment class, the less specific statements we can make 
- Different stationary stochastic settings: Normal versus Bernoulli, combinatorial?
- Adversarial bandits 

## Limitations of Bandit Algorithms 

- Choices today altering the available choices and rewards tomorrow (RL)
- Rewards are not always observable (partial monitoring) 
- Strategic agents (game theory) 

## Real World Applications

- A/B Testing
    - Adaptively selecting between new interfaces for a website (no need to end it!)
- Ad Placement
    - Is an ad clicked or not? 
    - User context 
- Recommendation Algorithms
    - Online learning versus matrix factorization
- Network or Route Planning
    - Time of shortest path
    - Similar paths may overlap; more intentional and efficient learning? 
- Dynamic Pricing
    - Never observe the true value, only know true value was higher than set price
- Tree Search
    - UCT algorithm (AlphaGO!)
    - Overlapping game decision paths